In [1]:
import pandas as pd
import numpy as np
import random
import pyttsx3
import ipywidgets as widgets
import csv
import numpy as np
import math as math
import statistics

In [2]:
mat=np.zeros((100,3))

In [3]:
save = np.array([1,2,3])

In [4]:
mat[0,]=save

In [5]:
mat[0,]

array([1., 2., 3.])

In [6]:
cov = [[1,1,1],
       [1,1,1],
       [1,1,1]]

In [7]:
cov

[[1, 1, 1], [1, 1, 1], [1, 1, 1]]

In [34]:
def collapsedgibbs3NormV(n, rho, x, y, z):
    mat = mat=np.zeros((n,3))
    mat[0, ] = np.array([x,y,z])
    for i in range(1,n):
        x = int(10*np.random.normal(rho * y, math.sqrt(rho+rho**3), 1))/10
        y = int(10*np.random.normal(rho * x, math.sqrt(rho+rho**3), 1))/10
        z = int(10*np.random.normal(rho * x + rho * y, math.sqrt(rho+rho**3), 1))/10
        mat[i, ] = [x*100,y,z]
    return(mat)

In [9]:
save = collapsedgibbs3NormV(2, 0.9, 1, 2, 3)

In [10]:
save

array([[1.0e+00, 2.0e+00, 3.0e+00],
       [1.9e+03, 1.8e+00, 1.6e+00]])

In [11]:
save[1,2]

1.6

In [12]:
math.sqrt(0.9 + 0.9**3)

1.2763228431709588

In [13]:
save = open("lyricsdata.csv")

In [14]:
csvreader = csv.reader(save)

In [15]:
header = []
header = next(csvreader)

In [16]:
df = pd.DataFrame(index=np.arange(8031), columns=np.arange(1))

In [17]:
rows = []
for row in csvreader:
        rows.append(row)

In [18]:
rows[2][0] = rows[2][0][1:]

In [19]:
str1 = ''.join(rows[2])

In [20]:
engine = pyttsx3.init()

In [21]:
engine.say(str1)
engine.runAndWait();

In [22]:
rate = engine.getProperty("rate")

In [23]:
voices = engine.getProperty("voices")

In [24]:
engine = pyttsx3.init()
engine.say('The quick brown fox jumped over the lazy dog.', 'fox')
engine.runAndWait();
engine.startLoop(False)
# engine.iterate() must be called inside externalLoop()
def externalLoop():
    for i in range(1, 3):
            engine = pyttsx3.init()
            rows[i][0] = rows[i][0][1:]
            str1 = ''.join(rows[i])  
externalLoop()
engine.endLoop()

In [35]:
def text_to_speech(numberOfLines = 1, startingPoint = 1, rho = 0.9, x = 300, y = 0, z = 0):
    engine = pyttsx3.init()
    engine.setProperty("volume", z)
    engine.setProperty("voice", voices[y].id)
    engine.setProperty("rate", x)
    if(z !=0):
        sample = collapsedgibbs3NormV(numberOfLines, rho, x, y, z)
        print(sample)
        for i in range(startingPoint, startingPoint + len(sample)):
            engine = pyttsx3.init()
            rows[i][0] = rows[i][0][1:]
            str1 = ''.join(rows[i])
            if(abs(sample[i-startingPoint,0]) >= 200 and abs(sample[i-startingPoint,0]) <= 1000):
                engine.setProperty("rate", abs(sample[i-startingPoint,0]))
            else:
                engine.setProperty("rate", 500)
#            engine.setProperty("voice", voices[int(sample[i-startingPoint,1])].id)
            if(abs(sample[i-startingPoint,2]) <= 1):
                engine.setProperty("volume", sample[i-startingPoint,2])
            else:
                engine.setProperty("volume", 0.5)
            engine.say(str1)
            engine.runAndWait(); 
            engine.stop()

In [26]:
text_to_speech(-400, -0.2, -47)

In [36]:
widgets.interact(text_to_speech, numberOfLines = [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                 startingPoint = (1,8020,1), rho =(-0.99,0.99,0.01), x= (-900, 900, 10), y = (-47,47,1), z = (-1,1,0.1));





interactive(children=(Dropdown(description='numberOfLines', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10…

In [28]:
# if needed:
# engine.endLoop()

In [2]:
import re
import wave
import pyaudio
import _thread
import time

class TextToSpeech:
    
    CHUNK = 1024

    def __init__(self, words_pron_dict:str = 'cmudict-0.7b.txt'):
        self._l = {}
        self._load_words(words_pron_dict)

    def _load_words(self, words_pron_dict:str):
        with open(words_pron_dict, 'r') as file:
            for line in file:
                if not line.startswith(';;;'):
                    key, val = line.split('  ',2)
                    self._l[key] = re.findall(r"[A-Z]+",val)

    def get_pronunciation(self, str_input):
        list_pron = []
        for word in re.findall(r"[\w']+",str_input.upper()):
            if word in self._l:
                list_pron += self._l[word]
        print(list_pron)
        delay=0
        for pron in list_pron:
            _thread.start_new_thread( TextToSpeech._play_audio, (pron,delay,))
            delay += 0.145
    
    def _play_audio(sound, delay):
        try:
            time.sleep(delay)
            wf = wave.open("sounds/"+sound+".wav", 'rb')
            p = pyaudio.PyAudio()
            stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                            channels=wf.getnchannels(),
                            rate=wf.getframerate(),
                            output=True)
            
            data = wf.readframes(TextToSpeech.CHUNK)
            
            while data:
                stream.write(data)
                data = wf.readframes(TextToSpeech.CHUNK)
        
            stream.stop_stream()
            stream.close()

            p.terminate()
            return
        except:
            pass
    
 
 

if __name__ == '__main__':
    tts = TextToSpeech()
    while True:
        tts.get_pronunciation(input('Enter a word or phrase: '))

ModuleNotFoundError: No module named 'pyaudio'